# Notebook du développement de BERTScore

Processus de développement du modèle BERTScore

In [5]:
import time
import pickle
from pathlib import Path
# !pip install gensim
import gensim
import pandas as pd
import numpy as np
import sklearn
import re
#from unidecode import unidecode
import functools
import operator
import psutil
from joblib import Parallel,delayed
from functools import partial
import time

In [7]:
# !pip install torch
import torch
import torch.nn as nn
from tqdm import tqdm

     |████████████████████████████████| 804.1 MB 4.1 kB/s  eta 0:00:01     |█████▊                          | 143.0 MB 14.0 MB/s eta 0:00:48     |██████▎                         | 158.4 MB 5.7 MB/s eta 0:01:54     |█████████▏                      | 231.4 MB 7.4 MB/s eta 0:01:18     |██████████████████████          | 554.4 MB 15.3 MB/s eta 0:00:17


In [8]:
embedding_section_W2V=pickle.load(open('embedding_section_W2V.pickle','rb'))
embedding_section_bert=pickle.load(open('embedding_section_bert.pickle','rb'))
print(type(embedding_section_bert),type(embedding_section_bert[0]),type(embedding_section_bert[0][0]))
len(embedding_section_bert)

<class 'list'> <class 'list'> <class 'torch.Tensor'>


405

In [17]:
print("Il y a ",len(embedding_section_bert),"articles")
section_mean=round(np.mean([len(i) for i in embedding_section_bert]),2)
print("avec en moyenne",section_mean,"phrases")
print("chaque phrase de dimension",embedding_section_bert[0][0].shape)

Il y a  405 articles
avec en moyenne 20.86 phrases
chaque phrase de dimension torch.Size([768])


In [46]:
ARTBERT=[torch.as_tensor([embedding_section_bert[k][i].tolist() for i in range(len(embedding_section_bert[k]))]) for k in range(len(embedding_section_bert))]
#ARTBERT=torch.as_tensor(ARTBERT)
len(ARTBERT),len(ARTBERT[0]),len(ARTBERT[0][0])

(405, 41, 768)

In [47]:
# On crée le Vecteur Sémantique Principal, proxy de l'idée principale du paragraphe
VSP=[i.mean(dim=0) for i in ARTBERT]
len(VSP),VSP[0].shape

(405, torch.Size([768]))

In [54]:
# Définition de la fonction de similarité
cos_sim=torch.nn.CosineSimilarity(dim=-1)
cos_sim(ARTBERT[0][:3],VSP[0])

tensor([0.9893, 0.9104, 0.9858])

In [55]:
# Tests
cos_sim=torch.nn.CosineSimilarity(dim=0)
cos_sim(ARTBERT[0][0],VSP[0]),cos_sim(ARTBERT[0][1],VSP[0]),cos_sim(ARTBERT[0][2],VSP[0])

(tensor(0.9893), tensor(0.9104), tensor(0.9858))

In [58]:
# Calcul de similarité de chaque phrase pour chaque paragraphe
cos_sim=torch.nn.CosineSimilarity(dim=-1)
similarite=[cos_sim(ARTBERT[i][:],VSP[i]) for i in range(len(VSP))]

In [65]:
# On récupère, pour chaque paragraphe, les phrases les plus similaires au proxy 
top_similarite=[i.topk(k=3)[1] if i.shape[0]>3 else torch.as_tensor([0,1,2]) for i in similarite]


In [66]:
pickle.dump(top_similarite,open("OrangeSum_BERT_Score.pickle",'wb'))